# SD-FEC LDPC Encoder/Decoder Loopback
This notebook provides a basic introduction to the SD-FEC IP core and how it is used for LDPC encoding and decoding. The hardware design comprises two parts in isolation - encoding and decoding. Two SD-FEC cores are set up, one to perform each operation, in a loopback system. This means that all data input to the cores is generated from software in this notebook. The reason for this is to illustrate the various data formats required at each stage of the design pipeline. Additionally, all control over the SD-FEC cores is executed from this notebook, again to provide the user with an understanding of the procedures involved when using the SD-FEC IP core. Therefore, this notebook and accompanying hardware design is not intended to demonstrate a high-throughput application, but instead provide a starting point for developing with SD-FEC.

## TOC:
* [Encoder](#encoder)
* [Modulation, Channel and Soft Demodulation](#Mod_CH_SDemod)
* [Decoder](#decoder)

Begin by downloading the hardware desing to the platform.

In [1]:
import numpy as np
from pynq import allocate
from pynq import Overlay
import random
import xsdfec
from uosfec import *

ol = Overlay('ldpc_loopback.bit')

## Encoder <a class="anchor" id="encoder"></a>
This first section covers the encoder component of the system. Data will be generated, output to the encoder where it will be encoded, and the resulting block will then be returned for analysis.

We can inspect the available LDPC codes that were loaded into the SD-FEC encoder IP core from Vivado.

In [2]:
fec = ol.ldpc_encoder.sd_fec
fec.available_ldpc_params()

['docsis_short',
 'docsis_medium',
 'docsis_long',
 'docsis_init_ranging',
 'docsis_fine_ranging',
 'wifi802_11_cr1_2_648',
 'wifi802_11_cr1_2_1296',
 'wifi802_11_cr1_2_1944',
 'wifi802_11_cr2_3_648',
 'wifi802_11_cr2_3_1296',
 'wifi802_11_cr2_3_1944',
 'wifi802_11_cr3_4_648',
 'wifi802_11_cr3_4_1296',
 'wifi802_11_cr3_4_1944',
 'wifi802_11_cr5_6_648',
 'wifi802_11_cr5_6_1296',
 'wifi802_11_cr5_6_1944',
 '5g_graph1_set1_l5_p32',
 '5g_graph1_set1_l5_p64',
 '5g_graph1_set1_l5_p128',
 '5g_graph1_set1_l5_p256',
 '5g_graph1_set2_l5_p384',
 '5g_graph1_set1_l46_p32',
 '5g_graph1_set1_l46_p64',
 '5g_graph1_set1_l46_p128',
 '5g_graph1_set1_l46_p256',
 '5g_graph1_set2_l46_p384',
 '5g_graph2_set1_l7_p32',
 '5g_graph2_set1_l7_p64',
 '5g_graph2_set1_l7_p128',
 '5g_graph2_set1_l7_p256',
 '5g_graph2_set2_l7_p384',
 '5g_graph2_set1_l42_p32',
 '5g_graph2_set1_l42_p64',
 '5g_graph2_set1_l42_p128',
 '5g_graph2_set1_l42_p256',
 '5g_graph2_set2_l42_p384']

Using DOCSIS Short as the example going forward, we can obtain useful information about the standard such as block length (n), information bits (k) and the sub-matrix size (p).

In [3]:
code_name = 'docsis_short'

n = fec._code_params.ldpc[code_name]['n']
k = fec._code_params.ldpc[code_name]['k']
p = fec._code_params.ldpc[code_name]['p']

print('Block Length: %s\nInformation Bits: %s\nSub-Matrix Size: %s' % (n, k, p))

Block Length: 1120
Information Bits: 840
Sub-Matrix Size: 56


### TX Data Generation

It can be seen that there are 840 information bits for every block. The data in port (DIN) of the SD-FEC block accepts data of the form uint8. Data can be concatenated together to form larger data buses to improve throughput i.e. 16 uint8 words can be grouped to create a 192 length word. For the purposes of this demonstration however, the number of words per transaction has been fixed to 1 uint8 word.

We will create two data buffers for sending data to and receiving data from the SD-FEC core. The TX buffer only contains the information bits and has a length of k/8 bytes. The RX buffer is of length n/8 bytes, the full block size. In addition to the information bits, the encoded RX buffer contains the **(Need to read up on what these actually are)** at the tail. 

Random data is generated and placed into the TX buffer.

In [4]:
tx_buffer = allocate(shape=(k//8,), dtype=np.uint32)
rx_buffer = allocate(shape=(n//8,), dtype=np.uint32)

for i in range(k//8):
    tx_buffer[i] = random.randint(1,255)

### Encoder Control Register

In addition to the data buffers, the SD-FEC core requires that a control register is written to for every block input to the core. If this register is not written to, the core will not output any data and drive tready low such that it blocks further data from being streamed into it. For non-5G LDPC codes, the wordlength of the control register is 32 bits and comprises:   

| Field                          | Type        |
| -----------                    | ----------- |
| External Block ID (31:24)      |  uint8      |
| Max No. Iterations (23:18)     |  uint6      |
| Terminate on no Change (17:17) |  bit1       |
| Terminate on Pass (16:16)      |  bit1       |
| Include Parity Output (15:15)  |  bit1       |
| Hard Output (14:14)            |  bit1       |
| Reserved (13:7)                |  uint7      |
| Code Number (6:0)              |  uint7      |

Of these fields, only two are used when the SD-FEC block is configured as an encoder. These are *External Block ID* and *Code Number*. 

The control register is written to over AXI4-Stream as opposed to an AXI4 memory map. This ensures that the control register is aligned with the block for high throughput applications. Therefore, a data buffer is used to transfer the required bits. A Python function has been created to take the fields and perform the neccessary bit packing to form the expected 32-bit word.

In [5]:
ctrl_buffer = allocate(shape=(1,), dtype=np.uint32)
status_buffer = allocate(shape=(1,), dtype=np.uint32)

codes = fec.available_ldpc_params()
code_number = codes.index(code_name)

id = code_number
code = code_number
ctrl_buffer[0] = non_5G_ldpc_ctrl_enc(id, code)

Non-5G NR Control Interface Definitions for LDPC Encode
	ID:  0
	CODE:  0


### Encoding Data
Two DMAs exist in the encoder section of the hardware design. One for sending and receiving the data and another for sending the control register and receiving information regarding the status.

In [6]:
dma_data = ol.ldpc_encoder.axi_dma_data
dma_ctrl = ol.ldpc_encoder.axi_dma_ctrl

Need to work out what *add_ldpc_params* and *63* do/mean????

In [8]:
fec.add_ldpc_params(0, 0, 0, 0, code_name)
fec.CORE_AXIS_ENABLE = 63

dma_ctrl.sendchannel.transfer(ctrl_buffer)
dma_data.sendchannel.transfer(tx_buffer)
dma_ctrl.recvchannel.transfer(status_buffer)
dma_data.recvchannel.transfer(rx_buffer)
dma_ctrl.sendchannel.wait()
dma_data.sendchannel.wait()
dma_data.recvchannel.wait()
dma_ctrl.recvchannel.wait()

For each block transaction, the core will return a status providing information regarding the code ID that was used for encoding the block as well as the type of output. For the encoder, the data output is always hard.

In [11]:
non_5G_ldpc_status_enc(status_buffer[0])

ID:  0
Hard OP:  1
OP:  1
Code:  0


The received data is inspected and compared with the transmitted data. The cell output below shows that the information buffer has been encoded and additional bits have been added to the end of the array (elements 106 to 140).

In [24]:
import pandas as pd
pd.set_option('display.max_rows', 500)

tx_frame = pd.DataFrame({'TX': tx_buffer})
rx_frame = pd.DataFrame({'RX': rx_buffer})

compare = pandas.concat([tx_frame, rx_frame], axis=1)
print(compare)

        TX   RX
0     95.0   95
1     25.0   25
2      5.0    5
3     25.0   25
4     11.0   11
5     24.0   24
6      3.0    3
7    131.0  131
8    147.0  147
9    206.0  206
10     6.0    6
11   143.0  143
12   201.0  201
13   129.0  129
14    65.0   65
15   207.0  207
16   214.0  214
17   152.0  152
18    67.0   67
19    68.0   68
20   121.0  121
21    96.0   96
22   214.0  214
23   148.0  148
24    14.0   14
25    89.0   89
26   218.0  218
27    92.0   92
28    52.0   52
29    64.0   64
30   121.0  121
31   133.0  133
32    41.0   41
33     6.0    6
34   182.0  182
35   110.0  110
36   202.0  202
37   125.0  125
38   116.0  116
39   246.0  246
40   130.0  130
41    86.0   86
42   142.0  142
43   199.0  199
44   211.0  211
45    98.0   98
46   151.0  151
47   100.0  100
48    32.0   32
49    88.0   88
50   121.0  121
51   115.0  115
52    75.0   75
53    76.0   76
54    28.0   28
55    34.0   34
56   166.0  166
57   250.0  250
58   179.0  179
59    30.0   30
60   216.0  216
61   122

## Modulation, Channel and Soft Demodulation <a class="anchor" id="Mod_CH_SDemod"></a>
This section will show how the encoded data is formatted for the decoder data input (DIN). The decoder only accepts soft data in the form of *Log-Likelihood Ratio (LLR)* values, meaning the encoded data has to first be modulated so that soft-decision demodulation can be performed to provide the neccessary input to the decoder. 

This has yet to be implemented in Python. For the time being export the encoded data to a \*.csv and perform the operations in MATLAB.

### Export Hard Data to \*.csv

In [33]:
np.savetxt('data.csv', rx_buffer, delimiter=',')

### Convert Hard Data to Soft Data
The MATLAB code for converting hard data to soft data is provided below:

```Matlab
block_size = 1120;
data  = readmatrix('data.csv');

binVal = decimalToBinaryVector(data,8);
binValCol = reshape(binVal',block_size,1);
binValColByteFlip = reshape(fliplr(binVal)', 1, block_size)';

qpskMod = comm.QPSKModulator('BitInput',true);
modData = qpskMod(binValColByteFlip);

demodLLR = comm.QPSKDemodulator('BitOutput',true,...
    'DecisionMethod','Log-likelihood ratio');

softDataLLRCol = demodLLR(modData);

writematrix(softDataLLRCol*-1,'data_soft.csv')
```

### Import Soft Data

In [34]:
with open('data_soft.csv') as f:
    soft_data = np.loadtxt(f, delimiter=",")

## Decoder <a class="anchor" id="decoder"></a>
The soft decision is performed on each bit, giving the likelihood of the bit being a 0 or a 1. Therefore the length of the TX buffer is the full block length (n) of the LDPC coding standard in bits. The decoder can either output soft or hard data. Here, we will be retreiving the information as decoded hard data, output as bytes, giving a length of k/8. 

In [124]:
tx_buffer = allocate(shape=(n,), dtype=np.uint32)
rx_buffer = allocate(shape=(k//8,), dtype=np.uint32)

### Formatting the Soft Data
The soft value log-likelihood ratio is defined as:

LLR(x) = ln(P(x=1) / P(x=0))

This means that positive values (including 0) are interpreted as a hard binary 1 and negative values are interpreted as a hard binary 0. The magnitude of these values indicates the certainty, with a larger magnitude equating to a higher certainty and vice versa.

The SD-FEC decoder accepts data in the form *fix6_2*. This data is symmetrically saturated meaning the range is from -7.75 to 7.75. Any values outside of this range are saturated to these maximum and minimum values. If too much saturation is occurring, the performance of the decoder may be affected. To avoid this, the data can be scaled by a number less than 1.

Additionally, this data must be sign extended to the nearest byte. Taking the limits as an example, the format

| Decimal Value | Binary Value | Unsigned Value |
| -----------   | -----------  | -----------    |
| 7.75          |  00011111    | 31             |
| -7.75         |  11100001    | 225            |

A Python function *format_llr* for formatting the data correctly is used to convert the soft data read from the \*.csv.

In [125]:
for i in range(n):
    tx_buffer[i] = format_llr(soft_data[i])

In [126]:
fec = ol.ldpc_decoder.sd_fec

id = 0
max_iter = 10
no_change = 0
term_on_pass = 0
parity = 0
hard = 1
code = 0
ctrl_buffer[0] = non_5G_ldpc_ctrl_dec(id, 
                                      max_iter, 
                                      no_change, 
                                      term_on_pass, 
                                      parity, 
                                      hard, 
                                      code)

Non-5G NR Control Interface Definitions for LDPC Decode
	ID:  0
	Max Iter:  10
	Term No Change:  0
	Term Pass:  0
	Parity Pass:  0
	Hard OP:  1
	Code:  0
	Binary:  00000000001010000100000000000000


In [127]:
fec = ol.ldpc_decoder.sd_fec

dma_data = ol.ldpc_decoder.axi_dma_data
dma_ctrl = ol.ldpc_decoder.axi_dma_ctrl

In [128]:
fec.add_ldpc_params(0, 0, 0, 0, code_name)
fec.CORE_AXIS_ENABLE = 63

dma_ctrl.sendchannel.transfer(ctrl_buffer)
dma_data.sendchannel.transfer(tx_buffer)
dma_ctrl.recvchannel.transfer(status_buffer)
dma_data.recvchannel.transfer(rx_buffer)
dma_ctrl.sendchannel.wait()
dma_data.sendchannel.wait()
dma_data.recvchannel.wait()
dma_ctrl.recvchannel.wait()

In [129]:
non_5G_ldpc_status_dec(status_buffer[0])

ID:  0 8
Dec Iter:  10 6
Term No Change:  0 1
Term Pass:  0 1
Parity Pass:  1 1
Hard OP:  1 1
Code:  0 8


In [130]:
rx_buffer

PynqBuffer([ 95,  25,   5,  25,  11,  24,   3, 131, 147, 206,   6, 143,
            201, 129,  65, 207, 214, 152,  67,  68, 121,  96, 214, 148,
             14,  89, 218,  92,  52,  64, 121, 133,  41,   6, 182, 110,
            202, 125, 116, 246, 130,  86, 142, 199, 211,  98, 151, 100,
             32,  88, 121, 115,  75,  76,  28,  34, 166, 250, 179,  30,
            216, 122, 113, 229, 127,  76,  44,  48, 206, 118,  99, 255,
            155, 223,   7, 184,  44,  34, 222, 209,  31,  48,  42, 208,
             86, 210, 212,  79, 245,  76, 189,   8, 124,  41, 153, 209,
             64, 247, 148, 245, 178, 150,  94,   3, 165], dtype=uint32)